Cleaner version of the notebook that I run on laptop to scape data faster

In [1]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os

In [2]:
#captcha alert settings
import winsound
duration = 5000  # milliseconds
freq = 440  # Hz

In [3]:
headers_list = [
# Firefox 77 Mac
{
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Firefox 77 Windows
{
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Chrome 83 Mac
{
"Connection": "keep-alive",
"DNT": "1",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "none",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
},
# Chrome 83 Windows 
{
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9"
}
]

In [4]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 5000
cities = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
                 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
                 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Boston',
                 'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose',
                 'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
                 'Memphis', 'Orlando', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
                 'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
                 'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
                 'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 
                 'Pittsburgh', 'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 
                 'Herndon','Reston', 'Roanoke']

In [7]:
i = 0
count = 0
results = []

for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        file = 'data_by_cities/'+str(city)+'.csv'
        try: 
            df_more = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
            print('file successfully opened')
        except:
            print('file not found, creating new file')
            df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
            df_more.to_csv(file, sep='\t', encoding='utf-8')
        datasize = df_more.shape
        url = url_template.format(city, start)
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        html = r.get(url)
        #html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        #captcha detector
        if 'hCaptcha solve page' in soup.get_text():
            print('captcha detected')
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobTitle').text.replace('\n', '').replace('new', '')
            except:
                title = 'None'
            try:
                location = each.find('div', {'class':"companyLocation" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='companyName').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('div', {'class': "metadata salary-snippet-container"}).text.replace('\n', '')
            except:
                try:
                    salary = each.find('span', {'class':'estimated-salary'}).text.replace('\n', '')
                except:
                    salary = 'None'
            synopsis = each.find('div', {'class':'job-snippet'}).text.replace('\n', '')
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            #drop duplicates
            df_more = df_more[~df_more.duplicated()].reset_index(drop=True)
        #count the number of pages scraped and print, so I know how far the code is running
        count+= 1
        print(count, ' loops done, proceed to sleep, current city is ', city)
        if df_more.shape == datasize:
            print(df_more.shape,' no new data, moving to next city')
            break       
        df_more.to_csv(file, sep='\t', encoding='utf-8')#write every page so i don't lose my data again
        sleep(random.randint(7,20))
        #I don't want too see long line of the process, clear_output will clear previous lines
        clear_output(wait=True)
        print(df_more.shape, ' data saved, last visited city is ', city, ', continuing....')
    print('moving to next city')
    sleep(random.randint(30,60))

(15, 5)  data saved, last visited city is  Las+Vegas , continuing....
file successfully opened
2  loops done, proceed to sleep, current city is  Las+Vegas


KeyboardInterrupt: 

In [8]:
df_more

,Title,Location,Company,Salary,Synopsis
0,Data Scientist,"Las Vegas, NV",Bally's Interactive,Estimated $83.2K - $105K a year,"Experience with data visualization tools, such..."
1,Experienced Mathematician (REMOTE),"Remote in Las Vegas, NV",Everi Games Inc,Estimated $89.6K - $113K a year,This position is responsible for writing and t...
2,Sr. Data Analyst,"Remote in Las Vegas, NV 89119",Aristocrat Technologies Inc,Estimated $96K - $122K a year,Mine multiple data sources and use data tools ...
3,Senior Mathematician,"Las Vegas, NV 89119+2 locations",Konami Gaming,Estimated $89.4K - $113K a year,401(k) plan with company match.Company paid Ho...
4,Senior Data Analyst - Engineering Analytics,"Las Vegas, NV 89118 (Enterprise area)",Verizon,"$70,000 - $129,000 a year",Using data visualization and other data-gather...
5,"Administrative Assistant 2, Department of Math...","Las Vegas, NV 89154 (Paradise area)","University of Nevada, Las Vegas","$33,972 a year","The main responsibilities of this job include,..."
6,Mathematician,"Hybrid remote in Las Vegas, NV",Light & Wonder,Estimated $63.5K - $80.3K a year,Welcome to the world of land-based gaming.Brin...
7,"VP, Global Analytics & Data Science","Las Vegas, NV 89119",Aristocrat Technologies Inc,Estimated $137K - $173K a year,"7+ years of experience in data analysis, data ..."
8,Artificial Intelligence or Big Data Engineer,"Las Vegas, NV",KTI - Kari Technologies International,Estimated $109K - $138K a year,"Base on the market demand, develop the core te..."
9,Kubernetes and Docker Cloud Developer,"Remote in Henderson, NV+1 location",Frontier Technology Inc.,Estimated $103K - $131K a year,Experience working with data stores/databases....
